In [7]:
import seaborn as sns
import metapack as mp
import pandas as pd
import numpy as np
import rowgenerators as rg
import matplotlib.pyplot as plt
from IPython.display import display 


import seaborn
import warnings
from zipfile import ZipFile
import json

import re
from pathlib import Path 


%matplotlib inline
sns.set_context('notebook')
mp.jupyter.init()


In [8]:
#pkg = mp.jupyter.open_package()
pkg = mp.jupyter.open_source_package()
pkg

# San Diego Homelessness Contract Annotations
`sdcta.org-hl_contracts-1.1.1` Last Update: 2021-08-11T20:09:03

_Processed TagTog annotations for homelessness contracts collected by the_


Processed TagTog annotations for homelessness contracts collected by the 
San Diego County Taxpayers Association.


 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)


## References
<ul><li> <strong>annotations</string>, <em>data/homelessness-contracts-20210811.zip</em>. Zip file downloaded from TagTog with text and annotations</li><ul>

In [9]:
!pip install -r requirements.txt -q
from flatten_json import flatten

In [12]:
az = pkg.reference('annotations_source').resolved_url.fspath
az

PosixPath('/Users/eric/proj/data-projects/text-classification/datasets/sdcta.org-hl_contracts/data/homelessness-contracts-20210811.zip')

In [16]:
rows = []
archive = ZipFile(az, 'r')
files = archive.namelist()


In [17]:
def mk_html_fn(e):
    """Make the HTML file name from the annotation file name"""
    return str(Path('homelessness-contracts/plain.html/pool/').joinpath(Path(e.replace('.ann.json','.plain.html')).name))


def munge(d, **kwargs):
    """Remove some field prefixes"""
    r = {}
    for k, v in d.items():
        r[re.sub(r'fields_f_(\d+)_', '', k) ] = v
        
    if 'confidence_who_0' in r:
        r['who'] = r['confidence_who_0']
        del r['confidence_who_0']
        
    if 'confidence_who_1' in r:
        r['who'] = r['confidence_who_1']
        del r['confidence_who_1']
        
    for k,v in kwargs.items():
        r[k] = v
        
    return r


for fn in  [fn for fn in archive.namelist() if fn.endswith('.ann.json')]:
    with archive.open(fn) as f:

        d = json.load(f)
        
       
        dr = [ munge(flatten(r), file_name=fn, html_file_name=mk_html_fn(fn)) for r in d['entities']]
       
        
        rows.extend(dr)
    
df = pd.DataFrame(rows)
    
# assign the annotation names from the classIds
with archive.open('homelessness-contracts/annotations-legend.json') as f:
    al = json.load(f)
    
df['anno_type'] = df.classId.replace(al)
    
col_map = {
    'offsets_0_start':'offset_start', 
    'offsets_0_text':'text',
    'confidence_state':'confidence'
    
}

df = df.rename(columns=col_map)
    
df.head().T
        
        

,0,1,2,3,4
classId,e_7,e_19,e_1,e_6,e_4
part,s1v1,s1v1,s1v1,s1v1,s1v1
offset_start,33,33,127,153,340
text,COUNTY OF SAN DIEGO,COUNTY OF SAN DIEGO- DEPARTMENT OF PURCHASINGA...,Interfaith Shelter Network,.,Statement ofWork
coordinates,[],NaN,[],[],[]
confidence,,pre-added,,pre-added,
confidence_prob,0.975784,1.0,0.813213,0.5,0.900681
fields,{},{},{},{},{}
normalizations,{},{},{},{},{}
who,ml:nalaf,user:SDTEF,ml:nalaf,ml:regex,ml:nalaf


In [23]:
# The columns 'part', 'offsets_0_start' and 'offsets_0_text' identify the text that was annotated in the
# HTML conversion of the PDF file. The 'part' is the HTML id for an element, and '_start' is the the character
# position of the text in the string conversion of that element. 

from IPython.display import Markdown, display
from bs4 import BeautifulSoup

def printmd(string):
    display(Markdown(string))


contexts = {}
for idx, r in df.iterrows():
    with archive.open(r.html_file_name, 'r') as f:
        
        if r.part not in contexts:
        
            st, en = r.offset_start,r.offset_start+len(r.text)
            html = f.read().decode('utf8')
            soup = BeautifulSoup(html, "html.parser")

            contexts[r.part] =  soup.find(id=r.part).string

contexts = pd.Series(contexts).to_frame()
contexts =contexts.rename(columns={0:'context'})
contexts.index.name = 'part'
contexts

,0,1,2,3,4
classId,e_7,e_19,e_1,e_6,e_4
part,s1v1,s1v1,s1v1,s1v1,s1v1
offset_start,33,33,127,153,340
text,COUNTY OF SAN DIEGO,COUNTY OF SAN DIEGO- DEPARTMENT OF PURCHASINGA...,Interfaith Shelter Network,.,Statement ofWork
coordinates,[],NaN,[],[],[]
confidence,,pre-added,,pre-added,
confidence_prob,0.975784,1.0,0.813213,0.5,0.900681
fields,{},{},{},{},{}
normalizations,{},{},{},{},{}
who,ml:nalaf,user:SDTEF,ml:nalaf,ml:regex,ml:nalaf


,context
part,
s1v1,SD CNTY PURCH '15 :fp 14PM04:12\n\nCOUNTY OF S...
s2v1,"COUNTY OF SAN DIEGO, HEALTH AND HUMAN SERVICES..."
s3v1,"COUNTY OF SAN DIEGO, HEALTH AND HUMAN SERVICES..."
s4v1,"COUNTY OF SAN DIEGO, HEALTH AND HUMAN SERVICES..."
s5v1,r---------------------------------------------...
...,...
s1p67,City of Poway— Development Services 13325 Civi...
s1p68,"Interfaith Community Services Greg Anglea, Exe..."
s1p71,City of Poway Attn: Darian Demerjian 13325 Civ...
